# Calculate RNA Secondary structure

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np
import forgi.graph.bulge_graph as fgb

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)
bed_folder = '{}/bed'.format(folder)

alt_exons_bedfile = '{}/exon2.bed'.format(bed_folder)
constitutive_bedfile = '{}/constitutive_exons.bed'.format(bed_folder)


splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

In [ ]:
figure_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_rna_properties'
! mkdir $figure_folder

prefix = 'isoform_transcriptions'
transcribed_fasta = '{}/{}.fa'.format(folder, prefix)

In [ ]:
from qtools import Submitter

rnafold_results = transcribed_fasta.replace('.fa', '_rnafold.txt')
command = 'RNAfold < {} > {}'.format(transcribed_fasta, rnafold_results)
print command
sub = Submitter([command], 'RNAfold', walltime='168:00:00', write_and_submit=True)

In [ ]:
! wc -l $transcribed_fasta

In [94]:
import re
from Bio import SeqIO

seriess = []

for record in SeqIO.parse('rnafold_output', 'fasta'):
    event_name, isoform = record.id.split('|')
    print record.seq
    s = str(record.seq).lstrip('ACGUT')
    search = re.search('([\+.(+)+]+)(\(-?\d+\.\d+\))', s)
    structure, mfe = search.groups()
    mfe = float(mfe.strip('()'))
    bg = fgb.BulgeGraph(dotbracket_str=structure)
    element_counts = pd.Series(Counter(bg.to_element_string()))
    element_counts = element_counts.rename(acronym_to_full)
    element_percentage = element_counts/element_counts.sum()
    element_percentage.index = element_percentage.index.map(lambda x: x + '_percentage')
    element_percentage['double_stranded_percentage'] = sum(1 for _ in structure if _ != '.')/float(len(s))
    element_percentage['minimum_free_energy'] = mfe
    element_percentage.index = element_percentage.index.map(lambda x: 'rnafold_'+ x)
    element_percentage['event_id'] = event_name
    element_percentage['isoform'] = isoform

    seriess.append(element_percentage)
structure_df = pd.DataFrame(seriess)
structure_df

CCACAGUGCCAGCUCCCUGCGCCCGGCCGACCUGCUUGCCCUCAUCCUCCUGGUUCAGGACCUCUACCCCAGCGAGAGCACAGCAGAGGACGACAUUCAGCCUUCCCCGCGGAGGGCCCGGAGCAGCCAGAACAUCCCCGUGCAGCAGGCCUGGAGCCCUCACUCCACGGGCCCAACUGGGGGGAGCUCUGCAGAGACGGAGACAGACAGCUUCUCCCUCCCUGAGGAGUACUUCACACCAGCUCCUUCCCCUGGCGAUCAGAGCUCAG..........(((((....((((.((((...((((((((.(((....(((((...))))).............))).))).))))).))..........(((((((....))))))).((((((..........(((((...(((..(((((((((......))))..)))))...))).)))))))))))(((.((.(((((.((....))))))).)).)))((((((...........))))))...)).))))....)))))...(-95.90)
CCACAGUGCCAGCUCCCUGCGCCCGGCCGACCUGCUUGCCCUCAUCCUCCUGGUUCAGGACCUCUACCCCAGCGAGAGCACAGCAGAGGACGACAUUCAGGAGACAGACAGCUUCUCCCUCCCUGAGGAGUACUUCACACCAGCUCCUUCCCCUGGCGAUCAGAGCUCAG..........(((((....((((.((((...((((((((.(((....(((((...))))).............))).))).))))).))..........(((((.((....)))))))......(((((((...........)))))))..)).))))....)))))...(-47.10)
GCUGCGGCCCCCAGACCUGGCGCAGCGUGUCCAGCUGUGGGAGCACUUCCAGAGCCUGCUGUGGACCUACAGCCGCCUGCGGGAGCAGGAGCAGUGCUUCGCCG

,event_id,isoform,rnafold_double_stranded_percentage,rnafold_five_prime_percentage,rnafold_hairpin_percentage,rnafold_interior_loop_percentage,rnafold_minimum_free_energy,rnafold_multiloop_percentage,rnafold_stem_percentage,rnafold_three_prime_percentage
0,exon:chr10:100190328-100190427:-@exon:chr10:10...,isoform2,0.555957,0.037175,0.104089,0.223048,-95.9,0.052045,0.572491,0.011152
1,exon:chr10:100190328-100190427:-@exon:chr10:10...,isoform1,0.483146,0.058824,0.105882,0.205882,-47.1,0.105882,0.505882,0.017647
2,exon:chr10:100193697-100193848:-@exon:chr10:10...,isoform2,0.644550,NaN,0.118644,0.096852,-183.1,0.111380,0.658596,0.014528


In [91]:
structure_df_2d = structure_df.pivot(index='event_id', columns='isoform')
structure_df_2d

rnafold_double_stranded_percentage  \
isoform                                                                      isoform1   
event_id                                                                                
exon:chr10:100190328-100190427:-@exon:chr10:100...                           0.483146   
exon:chr10:100193697-100193848:-@exon:chr10:100...                                NaN   

                                                              \
isoform                                             isoform2   
event_id                                                       
exon:chr10:100190328-100190427:-@exon:chr10:100...  0.555957   
exon:chr10:100193697-100193848:-@exon:chr10:100...  0.644550   

                                                   rnafold_five_prime_percentage  \
isoform                                                                 isoform1   
event_id                                                                           
exon:chr10:100190328-100190427:-@exon:chr10:100...                      0.058824   
exon:chr10:100193697-100193848:-@exon:chr10:100...                           NaN   

                                                              \
isoform                                             isoform2   
event_id                                                       
exon:chr10:100190328-100190427:-@exon:chr10:100...  0.037175   
exon:chr10:100193697-100193848:-@exon:chr10:100...       NaN   

                                                   rnafold_hairpin_percentage  \
isoform                                                              isoform1   
event_id                                                                        
exon:chr10:100190328-100190427:-@exon:chr10:100...                   0.105882   
exon:chr10:100193697-100193848:-@exon:chr10:100...                        NaN   

                                                              \
isoform                                             isoform2   
event_id                                                       
exon:chr10:100190328-100190427:-@exon:chr10:100...  0.104089   
exon:chr10:100193697-100193848:-@exon:chr10:100...  0.118644   

                                                   rnafold_interior_loop_percentage  \
isoform                                                                    isoform1   
event_id                                                                              
exon:chr10:100190328-100190427:-@exon:chr10:100...                         0.205882   
exon:chr10:100193697-100193848:-@exon:chr10:100...                              NaN   

                                                              \
isoform                                             isoform2   
event_id                                                       
exon:chr10:100190328-100190427:-@exon:chr10:100...  0.223048   
exon:chr10:100193697-100193848:-@exon:chr10:100...  0.096852   

                                                   rnafold_minimum_free_energy  \
isoform                                                               isoform1   
event_id                                                                         
exon:chr10:100190328-100190427:-@exon:chr10:100...                       -47.1   
exon:chr10:100193697-100193848:-@exon:chr10:100...                         NaN   

                                                             \
isoform                                            isoform2   
event_id                                                      
exon:chr10:100190328-100190427:-@exon:chr10:100...    -95.9   
exon:chr10:100193697-100193848:-@exon:chr10:100...   -183.1   

                                                   rnafold_multiloop_percentage  \
isoform                                                                isoform1   
event_id                                                                          
exon:chr10:100190328-100190427:-@exon:chr10:100...                     0.105882   

In [92]:
structure_df_2d.columns = ['_'.join(reversed(col)).strip() for col in structure_df_2d.columns.values]
structure_df_2d

,isoform1_rnafold_double_stranded_percentage,isoform2_rnafold_double_stranded_percentage,isoform1_rnafold_five_prime_percentage,isoform2_rnafold_five_prime_percentage,isoform1_rnafold_hairpin_percentage,isoform2_rnafold_hairpin_percentage,isoform1_rnafold_interior_loop_percentage,isoform2_rnafold_interior_loop_percentage,isoform1_rnafold_minimum_free_energy,isoform2_rnafold_minimum_free_energy,isoform1_rnafold_multiloop_percentage,isoform2_rnafold_multiloop_percentage,isoform1_rnafold_stem_percentage,isoform2_rnafold_stem_percentage,isoform1_rnafold_three_prime_percentage,isoform2_rnafold_three_prime_percentage
event_id,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,0.483146,0.555957,0.058824,0.037175,0.105882,0.104089,0.205882,0.223048,-47.1,-95.9,0.105882,0.052045,0.505882,0.572491,0.017647,0.011152
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,NaN,0.644550,NaN,NaN,NaN,0.118644,NaN,0.096852,NaN,-183.1,NaN,0.111380,NaN,0.658596,NaN,0.014528


In [84]:
s = '..........(((((....((((.((((...((((((((.(((....(((((...))))).............))).))).))))).))..........(((((((....))))))).((((((..........(((((...(((..(((((((((......))))..)))))...))).)))))))))))(((.((.(((((.((....))))))).)).)))((((((...........))))))...)).))))....)))))... (-95.90)'
structure, mfe = s.split()
structure

'..........(((((....((((.((((...((((((((.(((....(((((...))))).............))).))).))))).))..........(((((((....))))))).((((((..........(((((...(((..(((((((((......))))..)))))...))).)))))))))))(((.((.(((((.((....))))))).)).)))((((((...........))))))...)).))))....)))))...'

In [95]:
import forgi.graph.bulge_graph as fgb
structure = '..........(((((....((((.((((...((((((((.(((....(((((...))))).............))).))).))))).))..........(((((((....))))))).((((((..........(((((...(((..(((((((((......))))..)))))...))).)))))))))))(((.((.(((((.((....))))))).)).)))((((((...........))))))...)).))))....)))))...'
bg = fgb.BulgeGraph(dotbracket_str=structure)
bg.to_element_string()

'ffffffffffsssssiiiissssissssiiissssssssisssiiiissssshhhsssssiiiiiiiiiiiiisssisssisssssissmmmmmmmmmmssssssshhhhsssssssmssssssiiiiiiiiiisssssiiisssiissssssssshhhhhhssssiisssssiiisssissssssssssssssississsssisshhhhsssssssississssssssshhhhhhhhhhhssssssmmmssissssiiiisssssttt'

In [12]:
bg??

In [13]:
from pprint import pprint

In [32]:
from collections import Counter
element_counts = pd.Series(Counter(bg.to_element_string()))
element_counts.rename(acronym_to_full)

five_prime        10
hairpin           28
interior_loop     60
multiloop         14
stem             154
three_prime        3
dtype: int64

In [27]:
dict((s, bg.stem_length(s)) for s in bg.stem_iterator())

{'s0': 5,
 's1': 4,
 's10': 5,
 's11': 3,
 's12': 5,
 's13': 4,
 's14': 3,
 's15': 2,
 's16': 5,
 's17': 2,
 's18': 6,
 's2': 2,
 's3': 2,
 's4': 5,
 's5': 3,
 's6': 3,
 's7': 5,
 's8': 7,
 's9': 6}

In [10]:
ll -lha RNAfold.sh*

-rw-r--r-- 1 obotvinnik  404 Oct 20 15:22 RNAfold.sh
-rw------- 1 obotvinnik 4.0K Oct 21 21:37 RNAfold.sh.err
-rw------- 1 obotvinnik   24 Oct 21 21:41 RNAfold.sh.out


In [13]:
! tail RNAfold.sh.err

Warning from traverse_loop. Loop 18 has crossed regions
Warning from traverse_loop. Loop 18 has crossed regions
Warning from traverse_loop. Loop 6 has crossed regions
Warning from traverse_loop. Loop 8 has crossed regions
Warning from traverse_loop. Loop 147 has crossed regions
Warning from traverse_loop. Loop 142 has crossed regions
Warning from traverse_loop. Loop 7 has crossed regions
Warning from traverse_loop. Loop 7 has crossed regions
Warning from traverse_loop. Loop 2 has crossed regions
Warning from traverse_loop. Loop 24 has crossed regions


In [97]:
ls -lha $transcribed_fasta

-rw-r--r-- 1 obotvinnik yeo-group 40M Oct 20 11:28 /projects/ps-yeolab/obotvinnik/singlecell_pnms/isoform_transcriptions.fa


In [96]:
ls -lha $rnafold_results

-rw-r--r-- 1 obotvinnik yeo-group 2.2M Oct 20 15:59 /projects/ps-yeolab/obotvinnik/singlecell_pnms/isoform_transcriptions_rnafold.txt


mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_rna_properties': File exists


CCACAGUGCCAGCUCCCUGCGCCCGGCCGACCUGCUUGCCCUCAUCCUCCUGGUUCAGGACCUCUACCCCAGCGAGAGCACAGCAGAGGACGACAUUCAGCCUUCCCCGCGGAGGGCCCGGAGCAGCCAGAACAUCCCCGUGCAGCAGGCCUGGAGCCCUCACUCCACGGGCCCAACUGGGGGGAGCUCUGCAGAGACGGAGACAGACAGCUUCUCCCUCCCUGAGGAGUACUUCACACCAGCUCCUUCCCCUGGCGAUCAGAGCUCAG..........(((((....((((.((((...((((((((.(((....(((((...))))).............))).))).))))).))..........(((((((....))))))).((((((..........(((((...(((..(((((((((......))))..)))))...))).)))))))))))(((.((.(((((.((....))))))).)).)))((((((...........))))))...)).))))....)))))...(-95.90)


NameError: name 'acronym_to_full' is not defined

In [23]:
!head $transcribed_fasta

In [1]:
! RNAfold -h

RNAfold 2.1.8

Calculate minimum free energy secondary structures and partition function of 
RNAs

Usage: RNAfold [OPTIONS]...

The program reads RNA sequences from stdin, calculates their minimum free 
energy (mfe) structure and prints to stdout the mfe structure in bracket 
notation and its free energy. If the -p option was given it also computes the 
partition function (pf) and base pairing probability matrix, and prints the 
free energy of the thermodynamic ensemble, the frequency of the mfe structure 
in the ensemble, and the ensemble diversity to stdout.

It also produces PostScript files with plots of the resulting secondary 
structure graph and a "dot plot" of the base pairing matrix.
The dot plot shows a matrix of squares with area proportional to the pairing 
probability in the upper right half, and one square for each pair in the 
minimum free energy structure in the lower left half. For each pair i-j with 
probability p>10E-6 there is a line of the form

i  j  sqrt(p)  ubox

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_rna_properties': File exists


In [ ]:
! head $transcribed_fasta

^C


ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/obotvinnik/anaconda/lib/python2.7/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/home/obotvinnik/anaconda/lib/python2.7/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 260, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 212, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 391, in execute_request
    ident=ident)
  File "/home/obotvinnik/anaconda/lib/python2.7/site-packages/jupyter_client/session.py", line 645, in send
    if not isinstance(stream, zmq.Socket):
KeyboardInterrupt
E

In [31]:
pd.options.display.max_columns = 50

In [32]:
join_cols = gc_content_2d.columns.difference(splicing_feature_data.columns)

if len(join_cols) > 0:
    splicing_feature_data = splicing_feature_data.join(gc_content_2d[join_cols])
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exon4,exon4_length,gencode_id,gene_name,intron_length,junction_exons12,junction_exons13,junction_exons23,junction_exons24,junction_exons34,strand,exon2_divisible_by_3,one_ensembl_id,splice_type,exon2_merkin2012_ancient,isoform1_gc,isoform1_gc_position1,isoform1_gc_position2,isoform1_gc_position3,isoform2_gc,isoform2_gc_position1,isoform2_gc_position2,isoform2_gc_position3
event_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,NaN,NaN,ENSG00000107521.14,HPS1,928,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,True,ENSG00000107521,SE,False,75.714286,87.500000,77.551020,60.465116,77.292576,89.610390,73.076923,68.918919
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,NaN,NaN,ENSG00000107521.14,HPS1,3269,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,False,ENSG00000107521,SE,False,79.523810,88.311688,74.242424,74.626866,79.178886,73.333333,89.256198,73.913043
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,NaN,NaN,ENSG00000107521.14,HPS1,1543,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,False,ENSG00000107521,SE,False,78.111588,81.927711,73.239437,78.481013,75.882353,75.206612,53.684211,93.548387
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,NaN,NaN,ENSG00000120053.9,GOT1,2121,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,False,ENSG00000120053,SE,False,68.899522,62.318841,70.588235,73.611111,71.246006,70.642202,58.585859,83.809524
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,NaN,NaN,ENSG00000198018.6,ENTPD7,1857,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,False,ENSG00000198018,SE,False,74.111675,74.242424,76.271186,72.222222,70.833333,66.666667,71.276596,75.000000


In [33]:
splicing_feature_data.to_csv('{}/splicing_feature_data.csv'.format(folder))

## Calculate miRNA hybridization

For RNA targets, James Broughton from the Pasquinelli lab recommends `RNAhybrid`, and to use just the first 17 nt of the mature miRNA. Wanted to use `fastx-trimmer` but it only takes DNA sequences  - `U`'s are illegal :(

In [34]:
from Bio import SeqIO
import sys
import os

result_seq = []
filename = '/projects/ps-yeolab/genomes/mirbase/release_21/human_mature.fa'
with open(filename) as infile:
    for seq in SeqIO.parse(infile, 'fasta'):
        result_seq.append(seq[:17])

trimmed_filename = '/projects/ps-yeolab/genomes/mirbase/release_21/human_mature_17bp.fa'
with open(trimmed_filename, 'w') as outfile:
    SeqIO.write(result_seq, outfile, 'fasta')

Submit a compute job to calculate microRNA hybridization.

In [35]:
from gscripts.qtools import Submitter

mirna_seqs = '/projects/ps-yeolab/genomes/mirbase/release_21/human_mature_17bp.fa'
rnahybrid_results = '/projects/ps-yeolab/obotvinnik/miso_helpers/hg19/se_exon2_RNAhybrid_mirbase_human_mature_17bp.txt'
command = 'RNAhybrid -e -27 -c -s 3utr_human -q {} -t {} > {}'.format(mirna_seqs, transcribed_fasta, rnahybrid_results)
sub = Submitter([command], 'RNAhybrid', walltime='168:00:00', write_and_submit=True)

job ID: 3598068


Reading the output takes a LONG time

In [36]:
%%time
import pandas as pd
from collections import Counter

rnahybrid = pd.read_csv('/projects/ps-yeolab/obotvinnik/miso_helpers/hg19/se_exon2_RNAhybrid_mirbase_human_mature_17bp.txt', 
                        sep=':', 
#                         index_col=[0, 1, 3], 
                        header=None,
                        names=['chrom', 'start-stop', 'exon_length', 'mirna', 'mirna_length',
                                                                 'minimum_free_energy', 'p_value', 'target_bound_start', 'mirna_unbound',
                                                                 'mirna_bound', 'exon_bound', 'exon_bound'])
rnahybrid = rnahybrid.ix[rnahybrid.minimum_free_energy < -28]
grouped = rnahybrid.groupby(['chrom', 'start-stop'])
chrom_startstop_mirna = grouped.mirna.apply(lambda x: ','.join('{}[{}]'.format(k, v) for k,v in Counter(x).items()))
chrom_startstop_mirna.head()

CPU times: user 1.53 s, sys: 110 ms, total: 1.64 s
Wall time: 2.3 s
